In [102]:
import os
import time
import json
import random
import numpy as np
import pandas
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
print("completed imports")

completed imports


In [103]:
MODEL_EXPORT_PATH = "models"


In [104]:
def read_in_feature_from_json(file_path: str) -> list:
    with open(file_path, 'r') as f:
        feature = f.readlines()
        feature = json.loads(feature[0])
        class_label = feature[1]
        features = np.array(feature[0])
        feature = [features, class_label]
    return feature


def load_features(file_path: str) -> list:
    features = []
    start_time = time.time()
    for pd in os.listdir(file_path):
        full_parent_dir = os.path.join(file_path, pd)
        for file in (os.listdir(full_parent_dir)):
            features.append(read_in_feature_from_json(os.path.join(full_parent_dir, file)))
    end_time = time.time()
    print(f"Features loaded in {end_time - start_time}s")
    return features

In [105]:
MFCC_CLASS_LABEL_TUPLES_PATH = '../../data/segmented_songs/sample_dict_of_20_songs_0_segment_mfccs/'

labeled_data = load_features(MFCC_CLASS_LABEL_TUPLES_PATH)

# it is the assumption that python's random.shuffle is shuffling our data well enough
# train_test_split will also shuffle the data
random.shuffle(labeled_data)

Features loaded in 0.24408626556396484s


In [106]:
# create pandas DataFrame of mfcc array and class label pairs
featuresdf = pandas.DataFrame(labeled_data, columns=['feature', 'class_label'])

In [107]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
# note this is for use with categorical crossentropy per tf.keras.utils.to_categorical
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [108]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=127)

In [109]:
# model definition
num_labels = yy.shape[1]
filter_size = 2
model = Sequential()
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.001))

In [110]:
# begin model training
num_epochs = 500
num_batch_size = 64
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

Epoch 1/500
22/22 [==============================] - 1s 13ms/step - loss: 9.3854 - accuracy: 0.0941 - val_loss: 2.8879 - val_accuracy: 0.1567
Epoch 2/500
22/22 [==============================] - 0s 8ms/step - loss: 3.5317 - accuracy: 0.1540 - val_loss: 2.7078 - val_accuracy: 0.2991
Epoch 3/500
22/22 [==============================] - 0s 6ms/step - loss: 2.9874 - accuracy: 0.1768 - val_loss: 2.6088 - val_accuracy: 0.3504
Epoch 4/500
22/22 [==============================] - 0s

In [111]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 99.64%
Testing Accuracy: 96.87%


In [112]:
model_export_name = "FF_DNN_V_2.h5"
final_model_export_path = os.path.join(MODEL_EXPORT_PATH, model_export_name)

model.summary()

if not os.path.isfile(final_model_export_path):
    model.save(final_model_export_path)
    print(f"model saved to \"{final_model_export_path}\"")


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 256)               9216      
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_16 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)              